# Test estadísticos

### Descripción

<font size="4"> Comparación de algoritmos para minimización/maximización utilizando test estadísticos.
    
### Componentes
    
- <font size="3"> Se requiere un fichero csv separado para cada problema. En cada fichero csv, las columnas representan los algoritmos a comparar y las filas son las muestras de los resultados. Las cabeceras de todos los ficheros csv deben ser las mismas.
    
- <font size="3"> Se calculan ranking wins - loss para los siguientes test estadísticos:

    - <font size="3"> Mann-Whitney U test (Wilconxon rank-sum test), apropiado para la comparación  de algoritmos probabilistas ejecutados múltiples veces con distintas semillas (muestras independientes).

    - <font size="3"> Wilcoxon signed-rank test, apropiado para la comparación de algoritmos de aprendizaje evaluados en $k$-fold cross-validation, $n$ repeticiones (muestras pareadas).

- <font size="3"> Se calcula también el ranking wins - loss de los resultados medios.
    
---

## Qué es mejor, obtener la matriz de Wins and Losses a partir de resultados promedios o a partir de los test estadísticos?

La respuesta depende del nivel de confianza que quieras tener en la comparación de los algoritmos. Veamos las diferencias entre ambos enfoques y cuál es mejor.

### A partir de promedios
- No se considera la variabilidad en los datos.
    - si dos algoritmos tienen valores muy dispersos, la media puede ser engañosa.
    - Ejemplo: $AG1 = [0.9, 0.7]$, con media $0.8$; y $AG2 = [0.75, 0.75]$, con media $0.75$.
        - Aunque AG1 tiene una mejor media, su rendimiento NO es consistentemente mejor que AG2.
    - Entonces, ¿es realmente mejor AG1 que AG2? ¿O simplemete tuvo suerte en algunas ejecuciones? Por tanto, no sabemos si la diferencia es significativa.

### A partir de tests estadísticos
- Considera la variabilidad de los resultados.
    - Si un algoritmo tiene una media más alta pero mucha variabilidad, no se le da ventaja si las diferencias no son consistentes.
- Solo marca diferencias cuando son estadísticamente significativas.
    - Evita falsas afirmaciones como "AG1 es mejor que AG2", cuando en realidad la diferencia podría ser solo casualidad.
- Proporciona más información con la matriz de p-values.
    - Nos dice qué comparaciones son seguras y cuáles no tienen suficiente evidencia para concluir algo. Cuando el p-valor al comparar dos algoritmos es superior a 0.05, significa que no tenemos suficiente suficiente evidencia estadística para diferenciarlos, y que la diferencia que se pueda observar entre ambos podría ser simplemente azar.

### Requerimientos

In [ ]:
import numpy as np
import pandas as pd
import csv
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu
from statistics import mean

### Definición del tipo de test (maximización o minimización)

In [ ]:
def mejor(x,y):
    #return x>y # maximización
    return x<y # minimización

def peor(x,y):
    #return x<y # maximización
    return x>y # minimización

### Definición de la clase Ranking

In [ ]:
class Ranking:
    def __init__(self, name):
        self.name = name # Nombre del algoritmo
        self.wins = 0 # Veces que ha ganado
        self.losses = 0 # Veces que ha perdido
    def __lt__(self,x):
        return (self.wins-self.losses) < (x.wins-x.losses)
    def __str__(self):
        return f"{self.name:<15} {self.wins:>10} {self.losses:>10} {self.wins - self.losses:>15}"

### Calcula la matriz de wins - losses de las medias a partir de los scores de un problema

In [ ]:
def CalculateWinsLossesMatrixMean(scores):
    labels = scores.columns.values
    nScores = len(labels)
    WinLossMatriz = np.zeros((nScores,nScores))
    for i in range(nScores-1):
        score_i = scores.iloc[:,i].values
        for j in range(i+1,nScores):
            score_j = scores.iloc[:,j].values
            if mejor(mean(score_i),mean(score_j)):
                WinLossMatriz[i,j] = 1
                WinLossMatriz[j,i] = -1
            if peor(mean(score_i),mean(score_j)):
                WinLossMatriz[i,j] = -1
                WinLossMatriz[j,i] = 1
    return WinLossMatriz

### Calcula las veces que cada algoritmo ha ganado y ha perdido a partir de la matriz de wins - losses de un problema

In [ ]:
def CalculateWinsLossesAmount(WinsLossesMatriz,labels):
    nScores = len(labels)
    WinsLossesAmount = [Ranking(scoreName) for scoreName in labels]
    for i in range(nScores-1):
        for j in range(i+1,nScores):
            if WinsLossesMatriz[i,j]==1:
                WinsLossesAmount[i].wins += 1
                WinsLossesAmount[j].losses += 1
            if WinsLossesMatriz[i,j]==-1:
                WinsLossesAmount[j].wins += 1
                WinsLossesAmount[i].losses += 1
    return WinsLossesAmount

### Acumula las cantidades de wins - losses de un problema

In [ ]:
def AddWinsLossesAmount(WinsLossesTotalAmount,WinsLossesAmount):
    for i in range(len(WinsLossesAmount)):
        WinsLossesTotalAmount[i].wins += WinsLossesAmount[i].wins
        WinsLossesTotalAmount[i].losses += WinsLossesAmount[i].losses

### Calcula la matriz de wins - losses de las diferencias estadísticamente significativas para un test estadistico dado, junto con la matriz de p-values, a partir de los scores de un problema

In [ ]:
def CalculateWinsLossesMatrixStat(scores,stat):
    labels = scores.columns.values
    nScores = len(labels)
    WinLossMatriz = np.zeros((nScores,nScores))
    pValues = np.zeros((nScores,nScores))
    for i in range(nScores-1):
        score_i = scores.iloc[:,i].values
        for j in range(i+1,nScores):
            score_j = scores.iloc[:,j].values
            if not all(x_i == y_i for x_i, y_i in zip(score_i, score_j)): # si son datos diferentes
                _, p_value = stat(score_i, score_j)
                pValues[i,j] = p_value
                pValues[j,i] = p_value
                if p_value<0.05:
                    if mejor(mean(score_i),mean(score_j)):
                        WinLossMatriz[i,j] = 1
                        WinLossMatriz[j,i] = -1
                    if peor(mean(score_i),mean(score_j)):
                        WinLossMatriz[i,j] = -1
                        WinLossMatriz[j,i] = 1
    return WinLossMatriz,pValues

### Impresión de la matriz wins - losses

La siguiente funcion se ha creado para que, al imprimir la matriz de Wins and Losses, se interprete como que, si en una celda [i,j] hay "win", el algoritmo de la columna (j) gana al de la fila (i), a pesar de que las matrices se construyeron en los metodos anteriores con la lógica opuesta.

Esto es porque, segun esta implementada la función, y con razón, es mas facil imprimir primero la cabecera de la tabla (nombres de los algoritmos), es decir, las columnas que va  a tener la tabla. Y luego, se van añadiendo las filas una por una.

In [ ]:
def PrintMatriz(WinLossMatriz,labels):
    print("win: El algoritmo en la columna gana al algoritmo de la fila")
    print("loss: El algoritmo en la columna pierde frente al algoritmo de la fila")
    print("tie: El algoritmo en la columna empata con al algoritmo de la fila")
    n = len(labels)
    col_width = 10  # Ancho fijo para cada columna
    # Imprimir encabezados de columna
    print(" " * col_width, end="")
    for j in range(n):
        print(f"{labels[j]:>{col_width}}", end="")
    print()
    for i in range(n):
        print(f"{labels[i]:<{col_width}}", end="")
        for j in range(n):
            if i == j:
                print(f"{'-':>{col_width}}", end="")
            elif WinLossMatriz[j,i] == 1:
                print(f"{'win':>{col_width}}", end="")
            elif WinLossMatriz[j,i] == -1:
                print(f"{'loss':>{col_width}}", end="")
            else:
                print(f"{'tie':>{col_width}}", end="")
        print()

### Impresión  del ranking wins - losses

In [ ]:
def PrintRanking(WinLoss):
    Ranking = sorted(WinLoss, reverse=True)
    print(f"{'Ranking':<15} {'Wins':>10} {'Losses':>10} {'Wins-Losses':>15}")
    for r in Ranking:
        print(r)

### Impresión de la matriz de p-values

In [ ]:
def PrintPValuesMatriz(pValues, labels):
    n = len(labels)
    col_width = 15  # Ancho fijo para cada columna, ajusta según sea necesario
    print("p-values")
    # Imprimir encabezados de columna
    print(" " * col_width, end="")
    for j in range(n):
        print(f"{labels[j]:>{col_width}}", end="")
    print()

    # Imprimir filas con datos
    for i in range(n):
        print(f"{labels[i]:<{col_width}}", end="")
        for j in range(n):
            if i == j:
                print(f"{'-':>{col_width}}", end="")
            else:
                print(f"{pValues[i,j]:>{col_width}.8f}", end="")
        print()

### Realización de los test estadísticos e impresión de resultados

In [ ]:
fileName = ["gens"] # todas deben tener las mismas cabeceras de algoritmos
# En este ejemplo son sklearn.ensemble.RandomForestClassifier, sklearn.svm.SVC y sklearn.neural_network.MLPClassifier

scores = pd.read_csv(fileName[0]+".csv") # Se coje el primero de ellos para crear los objetos de la clase Ranking
labels = scores.columns.values # Nombres de los algoritmos
WinsLossesTotalAmountMean = [Ranking(scoreName) for scoreName in labels]
WinsLossesTotalAmountRanksum = [Ranking(scoreName) for scoreName in labels]
WinsLossesTotalAmountSignedRank = [Ranking(scoreName) for scoreName in labels]

for file in fileName:

    scores = pd.read_csv(file+".csv")
    print("file = ",file)
    print(scores)
    print('\n')

    # WINS-LOSSES DE MEDIAS
    # Matriz de dos dimensiones con valores -1, 0, 1 para comparar los algoritmos en cada problema
    WinsLossesMatrizMean = CalculateWinsLossesMatrixMean(scores)
    # Lista con los objetos Ranking de cada algoritmo
    WinsLossesAmountMean = CalculateWinsLossesAmount(WinsLossesMatrizMean,labels)
    # Acumula los wins-loss de los algoritmos en todos los problemas
    AddWinsLossesAmount(WinsLossesTotalAmountMean,WinsLossesAmountMean)
    print("Mean")
    PrintMatriz(WinsLossesMatrizMean,labels)
    PrintRanking(WinsLossesAmountMean)
    print('\n')

    # WINS-LOSSES Y P-VALUES DE RANK-SUM
    # Matriz de dos dimensiones con valores -1, 0, 1 para comparar los algoritmos en cada problema
    # y matriz de dos dimensiones para los p-values
    WinsLossesMatrizRanksum, pValuesRanksum = CalculateWinsLossesMatrixStat(scores,mannwhitneyu)
    # Lista con los objetos Ranking de cada algoritmo
    WinsLossesAmountRanksum = CalculateWinsLossesAmount(WinsLossesMatrizRanksum,labels)
    # Acumula los wins-loss de los algoritmos en todos los problemas
    AddWinsLossesAmount(WinsLossesTotalAmountRanksum,WinsLossesAmountRanksum)
    print("Ranksum")
    PrintMatriz(WinsLossesMatrizRanksum,labels)
    PrintPValuesMatriz(pValuesRanksum, labels)
    PrintRanking(WinsLossesAmountRanksum)
    print('\n')

    # WINS-LOSSES Y P-VALUES DE SIGNED-RANK
    # Matriz de dos dimensiones con valores -1, 0, 1 para comparar los algoritmos en cada problema
    # y matriz de dos dimensiones para los p-values
    WinsLossesMatrizSignedRank, pValuesSignedRank = CalculateWinsLossesMatrixStat(scores,wilcoxon)
    # Lista con los objetos Ranking de cada algoritmo
    WinsLossesAmountSignedRank = CalculateWinsLossesAmount(WinsLossesMatrizRanksum,labels)
    # Acumula los wins-loss de los algoritmos en todos los problemas
    AddWinsLossesAmount(WinsLossesTotalAmountSignedRank,WinsLossesAmountSignedRank)
    print("Signed Rank")
    PrintMatriz(WinsLossesMatrizSignedRank,labels)
    PrintPValuesMatriz(pValuesSignedRank, labels)
    PrintRanking(WinsLossesAmountSignedRank)
    print('\n')


print("Total Mean")
PrintRanking(WinsLossesTotalAmountMean)
print('\n')

print("Total Ranksum")
PrintRanking(WinsLossesTotalAmountRanksum)
print('\n')

print("Total Signed Rank")
PrintRanking(WinsLossesTotalAmountSignedRank)
print('\n')


file =  gens
    AE1  AE2
0  1000   66
1   690   67
2   908  327
3   727   74
4   630   72
5  1000   86
6   674   29
7  1000  100
8   950  297
9   705   81


Mean
win: El algoritmo en la columna gana al algoritmo de la fila
loss: El algoritmo en la columna pierde frente al algoritmo de la fila
tie: El algoritmo en la columna empata con al algoritmo de la fila
                 AE1       AE2
AE1                -       win
AE2             loss         -
Ranking               Wins     Losses     Wins-Losses
AE2                      1          0               1
AE1                      0          1              -1


Ranksum
win: El algoritmo en la columna gana al algoritmo de la fila
loss: El algoritmo en la columna pierde frente al algoritmo de la fila
tie: El algoritmo en la columna empata con al algoritmo de la fila
                 AE1       AE2
AE1                -       win
AE2             loss         -
p-values
                           AE1            AE2
AE1                       

Los warning se deben a que los p-values son extremedamente pequeños. El cambio a una aproximación normal ocurre porque, en pruebas exactas, como la prueba de Wilcoxon o la de rangos con signo, el cálculo del valor p se basa en la distribución exacta de los datos. Este tipo de pruebas asume ciertas condiciones en la distribución de los datos para que el cálculo sea preciso. Sin embargo, cuando los valores p resultan extremadamente pequeños, puede indicar que las diferencias entre grupos son demasiado grandes, lo que altera la precisión y estabilidad del cálculo exacto.

Al tener valores p tan bajos, el test puede asumir que el cálculo exacto no es viable o que sus resultados podrían ser inestables, por lo que automáticamente opta por una aproximación normal. Esta aproximación permite estimar el valor p basándose en una distribución normal, que es robusta y adecuada para conjuntos de datos grandes o con diferencias pronunciadas, y simplifica los cálculos en esos casos extremos.

Este cambio, entonces, asegura estabilidad y eficiencia en los resultados cuando las diferencias observadas son significativas y el valor p exacto cae en el rango de números extremadamente pequeños.